## Understanding and Predicting Property Maintenance Fines



Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [2]:


def blight_model():
    import pandas as pd
    import numpy as np
    from sklearn.model_selection import train_test_split
    import datetime
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.model_selection import GridSearchCV
    from sklearn.linear_model import LogisticRegression
    from sklearn import preprocessing
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import roc_curve, auc , roc_auc_score, precision_recall_curve

    
    
    dfTrain = pd.read_csv('train.csv', encoding = "ISO-8859-1", low_memory=False)
    dfTest = pd.read_csv('test.csv', encoding = "ISO-8859-1", low_memory=False)
    dfAddress = pd.read_csv('addresses.csv', encoding = "ISO-8859-1", low_memory=False)
    dfLatlons = pd.read_csv('latlons.csv', encoding = "ISO-8859-1", low_memory=False)
    
    
    dfEndereco =  pd.merge(dfLatlons, dfAddress, on='address')
    dfTrain = pd.merge(dfTrain,dfEndereco, on='ticket_id')
    dfTest = pd.merge(dfTest,dfEndereco, on='ticket_id')
    
    common_cols_to_drop = ['payment_amount','violation_code','address','disposition','country','agency_name','violation_description',
                       'state',
                       'zip_code','city','violation_street_name', 'violation_zip_code','mailing_address_str_number',
                       'mailing_address_str_name','payment_date', 'payment_status','collection_status',
                      'grafitti_status','inspector_name','violator_name', 'non_us_str_code', 'balance_due',]
    dfTrain = dfTrain.drop(common_cols_to_drop,axis=1).set_index("ticket_id")
    
    common_cols_to_drop2 = ['violation_code','address','disposition','country','agency_name','violation_description','state','zip_code','city','violation_street_name', 'violation_zip_code','mailing_address_str_number',
                       'mailing_address_str_name',
                      'grafitti_status','inspector_name','violator_name', 'non_us_str_code']
    dfTest = dfTest.drop(common_cols_to_drop2,axis=1).set_index("ticket_id")
    
    
    #convert string em date package of train
    data1 = pd.to_datetime(dfTrain['ticket_issued_date'])
    data2 = pd.to_datetime(dfTrain['hearing_date'])

    dfTrain['TimeDelta'] = data2 - data1
    dfTrain['TimeDelta'] = dfTrain['TimeDelta'].astype('int64')
    
    
    #convert string em date package of test proba
    data3 = pd.to_datetime(dfTest['ticket_issued_date'])
    data4 = pd.to_datetime(dfTest['hearing_date'])

    dfTest['TimeDelta'] = data4 - data3
    dfTest['TimeDelta'] = dfTest['TimeDelta'].astype('int64')
    
    
    dropis = ['ticket_issued_date','hearing_date']
    
    dfTest = dfTest.drop(dropis,axis=1)
    dfTrain = dfTrain.drop(dropis,axis=1)
    
    # mark zero values as missing or NaN
    dfTrain.fillna(0,inplace=True)
    # mark zero values as missing or NaN
    dfTest.fillna(0,inplace=True)
    
    
    #compliance [target variable for prediction] 
    # Nan = Not responsible =>not responsible by disposition ,not responsible by pending judgment disposition,
    # As this class is not subject to probability testing, we will delete it later

    # 0 = Responsible, non-compliant => non-compliant by no payment
    # 1 = Responsible, compliant => compliant by on-time payment ,'compliant by late payment within 1 month'

    # 1 replace NAN in compliance in another variable
    compliance_map = {'non-compliant by no payment' : 0,
                  'non-compliant by late payment more than 1 month' :0,
                  
                  'compliant by on-time payment' :1,
                  'compliant by early payment':1,
                  'compliant by no fine':1,
                  'compliant by payment with no scheduled hearing':1,
                  'compliant by payment on unknown date':1,
                  'compliant by late payment within 1 month': 1}

    dfTrain['compliance'] = dfTrain['compliance_detail'].map(compliance_map)
    
    
    
    # we found that there is data in the training without target, so let's delete it.
    dfTrain=dfTrain.replace('Nan',np.nan)
    dfTrain=dfTrain[~dfTrain[['compliance_detail','compliance']].isnull().all(axis=1)]
    
    
    # here we are deleting all lines whose classes will not be in the test data
    dfTrain = dfTrain[~(dfTrain.compliance.isnull())]
    
    
    drop_this = ['admin_fee','clean_up_cost','fine_amount','state_fee']
    
    dfTrain = dfTrain.drop(drop_this,axis=1)
    dfTest = dfTest.drop(drop_this,axis=1)
    
    
    y = dfTrain['compliance']
    X = dfTrain.drop('compliance', axis = 1)
    X = X.drop('compliance_detail', axis = 1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
    X_proba = dfTest
    
    

    scaler = MinMaxScaler()

    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)


    X_proba = scaler.transform(X_proba)
    
    
    

    grid_values = {'C': [ 0.1, 1, 10]}

    #Create the logistic regression object, 
    logistic = LogisticRegression( penalty  = 'l2')

    #Apply GridSearchCV to choose the best parameter with scoring 'roc_auc'
    grid_logistic = GridSearchCV(logistic, param_grid = grid_values,  scoring = 'roc_auc')

    #Train the model using the training data and check the score
    grid_logistic.fit(X_train, y_train)

    #Calculates the model score to measure auc
    y_score_log = grid_logistic.decision_function(X_test)

    #Predict the data contained in the test package
    y_pred_log = grid_logistic.predict(X_test)

    #Calculates the probability that a class will exit for each instance of the Test file
    probabilityLogistic = grid_logistic.predict_proba(X_proba)[:,1]

    #Best
    #print('Melhor score:', grid_logistic.best_score_)
    #print('Melhor parâmetro:',grid_logistic.best_params_)
    #print('Grid scores',grid_logistic.best_estimator_)

    #roc_auc
    fpr, tpr, _ = roc_curve(y_test, y_score_log)
    auc_score_log = auc(fpr,tpr)
    #print('AUC target score 0.7:','AUC score obtained:',auc_score_log)

    #Accuracy
    accuracy_log = accuracy_score(y_test, y_pred_log)
    #print("Accuracy: %.2f%%" % (accuracy_log * 100.0))
    
    
    result = pd.Series(probabilityLogistic, index = dfTest.index)




    
    return  result
    
blight_model()

# The target score was 0.70, I got the score 0,75 !!! Of course, 
# before I tried some other models and opted for the best in this case. 
# After choosing the best model for this specific case, I chose the best parameters.

(0.75773429804478576, ticket_id
 284932    0.070625
 285362    0.014396
 285361    0.084345
 285338    0.071368
 285346    0.086177
 285345    0.071369
 285347    0.094598
 285342    0.119250
 285530    0.014700
 284989    0.039613
 285344    0.094593
 285343    0.014735
 285340    0.014737
 285341    0.094602
 285349    0.086183
 285348    0.071374
 284991    0.039615
 285532    0.039868
 285406    0.039202
 285001    0.039955
 285006    0.014740
 285405    0.014386
 285337    0.038970
 285496    0.094810
 285497    0.071545
 285378    0.014380
 285589    0.039017
 285585    0.069940
 285501    0.086296
 285581    0.014369
             ...   
 376367    0.024120
 376366    0.064332
 376362    0.063412
 376363    0.069766
 376365    0.024120
 376364    0.064332
 376228    0.064848
 376265    0.063780
 376286    0.120946
 376320    0.064533
 376314    0.063798
 376327    0.123644
 376385    0.122814
 376435    0.172695
 376370    0.123626
 376434    0.092574
 376459    0.086012
 376478 